In [1]:
import requests
import json
import re
import time
import pandas as pd
from bs4 import BeautifulSoup
from discord import Webhook, RequestsWebhookAdapter

In [2]:
base_url = "https://www.google.com/finance/quote/"
discord_webhook_url = "https://discord.com/api/webhooks/940522536557498368/un_14JP-n7T6haQzIhsDB9Dfj_XjWB_2wYFK6oD-opeC-ZfRhgH29_LSbWnNGnOzGi5B"

In [12]:
def fetch_data(stock):
    final_url = base_url + stock
    data = {}
    html_data = requests.get(final_url).text
    soup = BeautifulSoup(html_data, "html.parser")
    data["title"] = soup.title.string

    current_price = soup.find('div', class_='YMlKec fxKbKc').text
    current_price = re.findall('', current_price)
    data['current_price'] = float(current_price[0])

    sidebar_data = soup.find('div', 'eYanAe').text
    print(sidebar_data)
    sidebar_data = re.findall('\d+\.\d+', sidebar_data)
    sidebar_data = [float(data) for data in sidebar_data]

    data['previous_close'] = sidebar_data[0]
    data['day_min'] = sidebar_data[1]
    data['day_max'] = sidebar_data[2]
    data['year_min'] = sidebar_data[3]
    data['year_max'] = sidebar_data[4]
    data['market_cap'] = sidebar_data[5]

    return data

In [13]:
data = fetch_data('INFY:NSE')
data

StockOwnership of a fraction of a corporation and the right to claim a share of the corporation's assets and profits equal to the amount of stock ownedClimate leaderCompany has received a CDP Climate Score of A, reflecting its commitment to environmental sustainability efforts, social impact, and corporate transparencyIN listed securityListed on NSEIN headquarteredHeadquartered in Bengaluru, Karnataka, IndiaPrevious closeThe last closing price₹1,715.00Day rangeThe difference between the high and low prices over the past day₹1,696.00 - ₹1,725.00Year rangeThe difference between the high and low prices over the past 52 weeks₹1,244.75 - ₹1,953.90Market capA valuation method that multiplies the price of a company's stock by the total number of outstanding shares.7.20T INRP/E ratioThe ratio of current share price to trailing twelve month EPS that signals if the price is high or low compared to other stocks33.61Dividend yieldThe ratio of annual dividend to current share price that estimates t

{'title': 'Infosys Ltd (INFY) Stock Price & News - Google Finance',
 'current_price': 708.6,
 'previous_close': 715.0,
 'day_min': 696.0,
 'day_max': 725.0,
 'year_min': 244.75,
 'year_max': 953.9,
 'market_cap': 7.2}

In [4]:
def send_meeage_to_discord(message):
    webhook = Webhook.from_url(discord_webhook_url, adapter=RequestsWebhookAdapter())
    webhook.send(message)

In [7]:
def update_config(data):
    with open('config.json', 'w') as f:
        json.dump(data, f)

def read_config():
    with open('config.json', 'r') as f:
        configs = json.load(f)
    
    return configs

In [9]:
while True:
    time.sleep(300)
    configs = read_config()
    
    for config in configs:
        stock_data = fetch_data(config['stock'])
        difference = config['min_price']- stock_data['current_price']
        if stock_data['current_price'] < config['min_price']:
            message = '{} is currently down at: {}, min threshold: {}, difference: {}'.format(
                                config['stock_name'], 
                                stock_data['current_price'],
                                config['min_price'], 
                                difference)
            config['min_price'] = stock_data['current_price']
            send_meeage_to_discord(message)
            update_config(configs)
            print(message)
        if stock_data['current_price'] > config['max_price']:
            difference = stock_data['current_price']- config['max_price']
            message = '{} is currently Up at: {}, min threshold: {}, difference: {}'.format(
                                config['stock_name'], 
                                stock_data['current_price'],
                                config['max_price'], 
                                difference)
            
            config['max_price'] = stock_data['current_price']
            send_meeage_to_discord(message)
            update_config(configs)
            print(message)

TRIDENT is currently down at: 60.15, min threshold: 61, difference: 0.8500000000000014


KeyboardInterrupt: 